# 🚀 IDD Models and Data - App Launcher

Welcome! This notebook provides an easy way to launch any of the available Shiny applications for epidemiological modeling.

## 🎯 Quick Start
1. **Run the first cell below** to discover available apps
2. **Run the second cell** to see the interactive launcher
3. **Select an app** from the dropdown menu
4. **Click "Launch App"** to start the server
5. **Access your app** at `/proxy/PORT/` (replace `/lab` with `/proxy/PORT/` in your browser URL)

## 📱 Available Applications

### 📊 Multi-Tab Dashboard
Complete interactive dashboard with multiple analysis tabs for comprehensive epidemiological modeling. Features parameter persistence across tabs and model comparisons.

### 🔬 SIR Model Demo  
Interactive demonstration of the classic SIR (Susceptible-Infectious-Recovered) epidemiological model. Perfect for learning and exploring basic disease dynamics.

### ⚖️ Model Comparison
Compare SIR, SEIR, and SEIRS models side by side. Visualize how different model structures affect disease progression predictions.

## 💡 Tips
- **Stop servers**: Use the "Stop Server" button or Kernel → Interrupt from the menu
- **Change ports**: Modify the port number if you need multiple apps running
- **Browser access**: Replace `/lab` with `/proxy/PORT/` in your browser URL to access the app

---
**Ready to get started? Run the cells below! ⬇️**

In [ ]:
# Setup and discovery
import sys
import os
from pathlib import Path
import importlib.util
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import threading

# Add src to Python path
src_dir = Path.cwd() / "src"
sys.path.insert(0, str(src_dir))

def discover_apps():
    """Discover all available Shiny apps."""
    apps_dir = src_dir / "idd_mad" / "apps"
    apps = {}
    
    if not apps_dir.exists():
        return apps
        
    # Known app descriptions
    descriptions = {
        'sir_demo': 'Interactive demonstration of SIR epidemiological model',
        'model_comparison': 'Compare SIR, SEIR, and SEIRS models side by side',
        'multi_tab_dashboard': 'Complete dashboard with multiple analysis tabs'
    }
        
    for app_dir in apps_dir.iterdir():
        if app_dir.is_dir() and not app_dir.name.startswith('__'):
            app_file = app_dir / "app.py"
            if app_file.exists():
                # Get description
                description = descriptions.get(app_dir.name, f"Shiny app: {app_dir.name}")
                
                apps[app_dir.name] = {
                    "name": app_dir.name.replace('_', ' ').title(),
                    "description": description,
                    "module": f"idd_mad.apps.{app_dir.name}.app",
                    "port": 8000 + len(apps)
                }
    
    return apps

# Discover apps
available_apps = discover_apps()

print(f"🔍 Found {len(available_apps)} apps:")
for app_id, app_info in available_apps.items():
    print(f"  📱 {app_info['name']} - {app_info['description']}")

if not available_apps:
    print("❌ No apps found. Make sure apps are in src/idd_mad/apps/")
else:
    print("\n✅ Ready to launch! Run the next cell to start the launcher.")

In [ ]:
# Interactive app selector
if available_apps:
    # Create dropdown options
    options = [(f"{info['name']} - {info['description']}", app_id) 
               for app_id, info in available_apps.items()]
    
    # Widgets
    app_selector = widgets.Dropdown(
        options=options,
        description='Select App:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px')
    )
    
    port_input = widgets.IntText(
        value=8000,
        description='Port:',
        min=8000,
        max=8999,
        layout=widgets.Layout(width='150px')
    )
    
    launch_button = widgets.Button(
        description='🚀 Launch App',
        button_style='success',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    stop_button = widgets.Button(
        description='⏹️ Stop Server',
        button_style='danger',
        layout=widgets.Layout(width='200px', height='40px'),
        disabled=True
    )
    
    output_area = widgets.Output()
    
    # Keep track of running server
    current_server = None
    
    def launch_app(button):
        nonlocal current_server
        
        with output_area:
            output_area.clear_output()
            
            app_id = app_selector.value
            port = port_input.value
            app_info = available_apps[app_id]
            
            print(f"🚀 Starting {app_info['name']}...")
            print(f"📱 Access at: /proxy/{port}/")
            print(f"   (Replace /lab with /proxy/{port}/ in your browser URL)")
            print("=" * 60)
            print("💡 Tip: Use the 'Stop Server' button or Kernel → Interrupt to stop")
            print()
            
            try:
                # Import and run the app
                module = __import__(app_info['module'], fromlist=['app'])
                
                if hasattr(module, 'app'):
                    launch_button.disabled = True
                    stop_button.disabled = False
                    
                    app = module.app
                    print(f"✅ Server starting on port {port}...")
                    app.run(host="0.0.0.0", port=port)
                else:
                    print(f"❌ No 'app' object found in {app_info['module']}")
                    
            except KeyboardInterrupt:
                print("\n🛑 Server stopped by user")
            except Exception as e:
                print(f"❌ Error starting app: {e}")
            finally:
                launch_button.disabled = False
                stop_button.disabled = True
                print("\n🔄 Ready to launch again!")
    
    def stop_server(button):
        """Stop the running server."""
        with output_area:
            print("\n🛑 Stopping server...")
            # Note: In Jupyter, use Kernel → Interrupt to stop the server
            print("💡 Use Kernel → Interrupt from the menu to stop the server")
    
    launch_button.on_click(launch_app)
    stop_button.on_click(stop_server)
    
    # Display widgets
    display(widgets.VBox([
        widgets.HTML("<h3>🎯 App Launcher</h3>"),
        app_selector,
        widgets.HBox([port_input, widgets.HTML("<div style='width: 20px;'></div>")]),
        widgets.HBox([launch_button, stop_button]),
        widgets.HTML("<hr>"),
        output_area
    ]))
    
else:
    display(widgets.HTML("<h3>❌ No apps found</h3><p>Make sure apps are in src/idd_mad/apps/</p>"))